In [9]:
# importing required libraries/modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import pickle
import json

In [10]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,0,1,1,0,0,1,2174,0,40,1,0
1,50,1,1,0,2,0,1,0,0,13,1,0
2,38,2,0,1,0,0,1,0,0,40,1,0
3,53,2,0,0,0,1,1,0,0,40,1,0
4,28,2,1,0,2,1,0,0,0,40,0,0


In [27]:
for col in ['workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'native_country', 'income']:
    train[col] = train[col].astype('str')

In [95]:
for col in ['workclass', 'education', 'marital_status', 'occupation', 'race', 'sex', 'native_country', 'income']:
    test[col] = test[col].astype('category')

In [14]:
y = train.income
X = train.drop(['income'], axis=1)
X.head()

,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,39,0,1,1,0,0,1,2174,0,40,1
1,50,1,1,0,2,0,1,0,0,13,1
2,38,2,0,1,0,0,1,0,0,40,1
3,53,2,0,0,0,1,1,0,0,40,1
4,28,2,1,0,2,1,0,0,0,40,0


In [20]:
y.shape[0]

32561

In [21]:
type(['sd'])

list

In [30]:
type({"sda": "dsd"})

dict

In [28]:
train.race.dtype

dtype('O')

In [8]:
X.native_country.unique()

NameError: name 'X' is not defined

In [97]:
y_test = test.income
X_test = test.drop(['income'], axis=1)

In [59]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, 
                                                  random_state=42, stratify = y)

In [60]:
# ohe = OneHotEncoder(handle_unknown='ignore')
# ohe.fit(X_train)

# X_train = ohe.transform(X_train)
# X_valid = ohe.transform(X_valid)
# X_test = ohe.transform(X_test)

# X_train = pd.DataFrame(X_train.todense())
# X_valid = pd.DataFrame(X_valid.todense())
# X_test = pd.DataFrame(X_test.todense())

In [67]:
parameters = {'max_depth': np.arange(2, 10),
                     'min_samples_split': np.arange(2, 6)}
clf_rf = RandomizedSearchCV(RandomForestClassifier(n_estimators=200, random_state=42), 
                            parameters, cv=10, n_iter=10)
clf_rf.fit(X_train, y_train)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]), 'min_samples_split': array([2, 3, 4, 5])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [68]:
clf_rf.best_params_

{'min_samples_split': 4, 'max_depth': 8}

In [31]:
model = RandomForestClassifier(n_estimators=200, max_depth=8, min_samples_split=4)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
model.score(X_val, y_val)

0.8564847988535162

In [112]:
model.score(X_test, y_test)

0.8579325594250967

In [119]:
with open('columns.json', 'w') as fh:
    json.dump(X.columns.tolist(), fh)

In [120]:
with open('dtypes.pickle', 'wb') as fh:
    pickle.dump(X.dtypes, fh)

In [123]:
from sklearn.externals import joblib
joblib.dump(model, 'model.pickle')

['model.pickle']

In [79]:
with open('columns.json', 'r') as fh:
    columns = json.load(fh)

In [80]:
with open('dtypes.pickle', 'rb') as fh:
    dtypes = pickle.load(fh)

In [81]:
model = joblib.load('model.pickle')

In [88]:
new_obs_str = '{"age": 34, "workclass": 1, "education": 2, "marital_status": 1, "occupation": 1, "race" : 1, "sex": 0, "capital_gain": 123, "capital_loss": 0, "hours_per_week": 45, "native_country": 2}'
                                            

In [89]:
new_obs_dict = json.loads(new_obs_str)
obs = pd.DataFrame([new_obs_dict], columns=columns)
obs = obs.astype(dtypes)

In [90]:
obs

,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,34,1,2,1,1,1,0,123,0,45,2


In [91]:
outcome = model.predict_proba(obs)
outcome

array([[0.69257417, 0.30742583]])

In [7]:
type(X)

NameError: name 'X' is not defined

In [92]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22792 entries, 24029 to 5800
Data columns (total 11 columns):
age               22792 non-null int64
workclass         22792 non-null int64
education         22792 non-null int64
marital_status    22792 non-null int64
occupation        22792 non-null int64
race              22792 non-null int64
sex               22792 non-null int64
capital_gain      22792 non-null int64
capital_loss      22792 non-null int64
hours_per_week    22792 non-null int64
native_country    22792 non-null int64
dtypes: int64(11)
memory usage: 2.1 MB


In [116]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 11 columns):
age               32561 non-null int64
workclass         32561 non-null category
education         32561 non-null category
marital_status    32561 non-null category
occupation        32561 non-null category
race              32561 non-null category
sex               32561 non-null category
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null category
dtypes: category(7), int64(4)
memory usage: 1.2 MB
